# Making a slim AR6 WG3 config file

Theme Song: Major System Error<br>
Artist: Marmozets<br>
Album: Knowing What You Know Now<br>
Released: 2018

the original WG3 json is 68 MB, but it's mostly repetition, so let's make a smaller file.

In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
accept_inds = np.loadtxt('../data_output_large/fair-samples/accept_inds.csv', dtype=int)
accept_inds

array([   156,    358,    441, ..., 998289, 998908, 999021])

## Save arg lists that pass constraint for running openscm-runner

In [3]:
# ozone
ozone_feedback = np.load('../data_input_large/fair-samples/ozone_feedback_unconstrained.npy')
beta_ch4 = np.load('../data_input_large/fair-samples/beta_ch4_unconstrained.npy')
beta_n2o = np.load('../data_input_large/fair-samples/beta_n2o_unconstrained.npy')
beta_ods = np.load('../data_input_large/fair-samples/beta_ods_unconstrained.npy')
beta_co = np.load('../data_input_large/fair-samples/beta_co_unconstrained.npy')
beta_voc = np.load('../data_input_large/fair-samples/beta_voc_unconstrained.npy')
beta_nox = np.load('../data_input_large/fair-samples/beta_nox_unconstrained.npy')

# carbon cycle
r0 = np.load('../data_input_large/fair-samples/r0_unconstrained.npy')
rC = np.load('../data_input_large/fair-samples/rC_unconstrained.npy')
rT = np.load('../data_input_large/fair-samples/rT_unconstrained.npy')
pre_ind_co2 = np.load('../data_input_large/fair-samples/pre_ind_co2_unconstrained.npy')

# aerosol
beta_so2 = np.load('../data_input_large/fair-samples/beta_so2_unconstrained.npy')
beta_bc = np.load('../data_input_large/fair-samples/beta_bc_unconstrained.npy')
beta_oc = np.load('../data_input_large/fair-samples/beta_oc_unconstrained.npy')
beta_nh3 = np.load('../data_input_large/fair-samples/beta_nh3_unconstrained.npy')
beta = np.load('../data_input_large/fair-samples/beta_unconstrained.npy')
aci_coeffs = np.load('../data_input_large/fair-samples/aci_coeffs.npy')

# forcing
scale_normals = np.load('../data_input_large/fair-samples/scale_normals.npy')
trend_solar = np.load('../data_input_large/fair-samples/scale_trend_solar.npy')

# climate response
geoff_sample_df = pd.read_csv('../data_output_large/geoff_sample.csv')
f2x = np.load('../data_input_large/fair-samples/f2x_unconstrained.npy')

# add in natural emissions and natural forcing: go to 2110 for WG3
ch4_n2o_df = pd.read_csv('../data_output/fair_wg3_natural_ch4_n2o.csv')
ch4_n2o = ch4_n2o_df.values[:361,1:]

df = pd.read_csv('../data_output/solar_erf.csv', index_col='year')
solar_forcing = df.solar_erf.loc[1750:2110].values

df = pd.read_csv('../data_output/volcanic_erf.csv', index_col='year')
volcanic_forcing = np.zeros((361))
volcanic_forcing[:269] = df.volcanic_erf.loc[1750:2018].values
# ramp down last 10 years to zero according to https://www.geosci-model-dev.net/9/3461/2016/gmd-9-3461-2016.html
volcanic_forcing[268:279] = volcanic_forcing[268] * np.linspace(1,0,11)
volcanic_forcing[279:] = 0.

In [4]:
common_configs = {
    'cfc11_adj': 1.13,
    'cfc12_adj': 1.12,
    'default_solar': solar_forcing.tolist(),
    'default_volcanic': volcanic_forcing.tolist(),
    'C_pi': [731.406, 273.8651, 34.05, 0.025, 0.004447, 5.3, 457],
    'C_pi_ind': [1, 2, 3, 20, 25, 29, 30],
    'E_pi': [1.22002422, 348.527359, 60.0218262, 3.87593407, 2.09777075, 15.4476682, 6.92769009],
    'aCO2land': -0.2/190,
    'E_ref_BC': 6.095,
    'ghg_forcing':'Meinshausen',
    'stwv_from_ch4': 0.079047,
    'F_ref_BC': 0.08,
    'E_ref_BC': 6.095,
    'tropO3_forcing': 'thornhill-skeie',
    'natural': ch4_n2o.tolist(),
}
with open('../data_output_large/fair-samples/fair-1.6.2-wg3-params-common.json', 'w') as filehandle:
    json.dump(common_configs, filehandle)

In [5]:
common_configs

{'cfc11_adj': 1.13,
 'cfc12_adj': 1.12,
 'default_solar': [0.09759026199288964,
  0.07924397293038964,
  0.04916604324288964,
  0.012988284453827144,
  -0.021426608124297853,
  -0.04590392746023535,
  -0.05348581222586035,
  -0.034343893280547856,
  0.017013235625702144,
  0.06751233230538964,
  0.09239526687570214,
  0.08624863601632714,
  0.06415820632882714,
  0.03081975418038964,
  -0.005935226288360353,
  -0.03577914718679785,
  -0.049897677460235365,
  -0.03671518234304785,
  0.019446927031952145,
  0.08559341140695212,
  0.11726260086007713,
  0.10971191726632716,
  0.08677905593820212,
  0.05083530593820214,
  0.010086575469452146,
  -0.025638766327422853,
  -0.04679316085867285,
  -0.04298661788992285,
  0.0016466584772646454,
  0.07305054031320214,
  0.11582734695382715,
  0.11567134109445212,
  0.09138122878976464,
  0.05451704421945215,
  0.010991409453827143,
  -0.02570116867117285,
  -0.04499909347586035,
  -0.03130177902273535,
  0.03730959793038965,
  0.1174498078913271

In [6]:
config_list = []
for ens in accept_inds:
    scale = np.ones(9)
    scale[0] = scale_normals[ens,1] * 0.86 # methane adjustment
    scale[1] = scale_normals[ens,2] * 1.07
    scale[2] = scale_normals[ens,3]
    scale[3:5] = scale_normals[ens,5:7]
    scale[5:9] = scale_normals[ens,7:11]
    fair_params = {
        'scale': scale.tolist(),
        'trend_solar': trend_solar[ens],
        'C_pi_CO2': pre_ind_co2[ens],
        'F2x' : f2x[ens],
        'r0'  : r0[ens],
        'rt'  : rT[ens],
        'rc'  : rC[ens],
        'lambda_global': -geoff_sample_df.loc[ens, 'lamg'],
        'ocean_heat_capacity':[geoff_sample_df.loc[ens, 'cmix'], geoff_sample_df.loc[ens, 'cdeep']],
        'ocean_heat_exchange':geoff_sample_df.loc[ens, 'gamma_2l'],
        'deep_ocean_efficacy':geoff_sample_df.loc[ens,'eff'],
        'b_aero': [beta_so2[ens], beta_bc[ens], beta_oc[ens], beta_nh3[ens]],
        'ghan_params':[beta[ens], aci_coeffs[ens,0], aci_coeffs[ens, 1]],
        'b_tro3': [beta_ch4[ens], beta_n2o[ens], beta_ods[ens], beta_co[ens], beta_voc[ens], beta_nox[ens]],
        'ozone_feedback': ozone_feedback[ens],

    }
    config_list.append(fair_params)
with open('../data_output_large/fair-samples/fair-1.6.2-wg3-params-slim.json', 'w') as filehandle:
    json.dump(config_list, filehandle)

In [7]:
len(config_list)

2237